In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; float:center}</style>")

In [8]:
from __future__ import division, print_function

import os

import pandas as pd
import numpy as np
import re
# import matplotlib.pyplot as plt
# import seaborn as sns

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

porter = PorterStemmer()

# %matplotlib inline

# sns.set_context('notebook')
# sns.set_style('ticks')

### set styles, stopwords, define functions

In [9]:
stops = set(stopwords.words('english'))
    # using a set will make it faster to run through...

punctuation = ['.',',',':','!',';','-','?','"',"'",'(',')','—']   
other = ['ive','ve', "i've", "i'v", 'i’ll', 'i’ve', 'i’v']  # 'deb','hideb','don','didn','twaittry','doesn','thank','heydeb',
    
stops_punc = set(stopwords.words('english') + punctuation)

mystops = stopwords.words('english') + punctuation + other
mystops_set = set(stopwords.words('english') + punctuation + other)

In [10]:
def tokenize_sentences(comment):
    return nltk.sent_tokenize(remove_newlines(comment.lower()))
    
    
# def separate_sentences(frame, identifier, paragraph, how='merge'):
#     sentences = pd.concat([pd.Series(row[identifier], tokenize_sentences((row[paragraph]))) for _, row in frame.iterrows()]).reset_index()
#     sentences.columns = ['sentence', identifier]
    
#     if how == 'merge':
#         return sentences.merge(frame, left_on=identifier, right_on=identifier, how='outer', sort=False, suffixes=('_l','_r')) 
#     elif how == 'nomerge': 
#         return sentences
#     else: 
#         return sentences.merge(frame, left_on=identifier, right_on=identifier, how='outer', sort=False, suffixes=('_l','_r')) 

# slightly faster version
def separate_sentences(frame, identifier, paragraph, how='merge'):
    sentences = pd.DataFrame((tokenize_sentences(row[paragraph]) for _, row in frame.iterrows()), index=frame[identifier]).stack()
    sentences = sentences.reset_index() [[0, identifier]] # var1 variable is currently labeled 0
    sentences.columns = ['sentence', identifier] # renaming var1
    
    if how == 'merge':
        return sentences.merge(frame, left_on=identifier, right_on=identifier, how='outer', sort=False, suffixes=('_l','_r')) 
    elif how == 'nomerge': 
        return sentences
    else: 
        return sentences.merge(frame, left_on=identifier, right_on=identifier, how='outer', sort=False, suffixes=('_l','_r')) 


def make_lowercase(comment):
    return remove_newlines(comment.lower())


# def tokenize_aslist(comment):
#     comment = remove_newlines(comment)
#     if comment == []:
#         return None
#     else: 
#         return [word for word in word_tokenize(remove_newlines(comment).lower()) if word not in mystops]

    
def tokenize(comment):
    comment = remove_newlines(comment)
    if comment == []:
        return None
    else: 
        return ' '.join([word for word in word_tokenize(remove_newlines(comment).lower()) if word not in mystops])

def ngram(comment):
    comment = remove_newlines(comment)
    if comment == []:
        return None
    else: 
        return [word for word in ngrams(remove_newlines(comment).lower().split(),gram) if word not in mystops]

    
# def tokenize_stem_aslist(comment):
#     tokens = word_tokenize(remove_newlines(comment).lower())
#     if tokens == []:
#         return None
#     else: 
#         return[porter.stem(word) for word in tokens if word not in mystops]
    
def tokenize_stem(comment):
    tokens = word_tokenize(remove_newlines(comment).lower())
    if tokens == []:
        return None
    else: 
        return ' '.join([porter.stem(word) for word in tokens if word not in mystops])


def remove_newlines(comment):    
    return re.sub(r"\n", " ", comment)



def preprocess_comments_data(frame):
    # make sure commentIDs are unique ( = row identity)
    frame.loc[:,'commentID'] = frame.index

    # remove any frame with no comment text
    frame = frame.loc[pd.notnull(frame['usercomment']),:]

    # replace NaN usernames with 'anon'
    frame.loc[:,'username'].fillna('anon', inplace=True)

    # tokenize data
    frame.loc[:,'usercomment'] = frame.loc[:,'usercomment'].apply(remove_newlines)
    frame.loc[:,'usercomment_lower'] = frame.loc[:,'usercomment'].apply(make_lowercase)

    frame.loc[:,'tokens'] = frame.loc[:,'usercomment'].apply(tokenize)
    frame.loc[:,'tokens_stemmed'] = frame.loc[:,'usercomment'].apply(tokenize_stem)
    
    frame.dropna(inplace=True)
    frame.drop([],axis=0, inplace=True)

    
    frame2 = separate_sentences(frame, 'commentID','usercomment',how='merge')
    frame2.dropna(inplace=True)
    frame2.drop([],axis=0, inplace=True)
    

    frame2.loc[:,'sentence_tokens'] = frame2.loc[:,'sentence'].apply(tokenize)
    frame2.loc[:,'sentence_tokens_stemmed'] = frame2.loc[:,'sentence'].apply(tokenize_stem)
    
    gram = 2
    comments_classified.loc[:,'sentence_bigrams'] = comments_classified.loc[:,'sentence'].apply(ngram)
    gram = 3
    comments_classified.loc[:,'sentence_trigrams'] = comments_classified.loc[:,'sentence'].apply(ngram)
    
    frame2.dropna(inplace=True)
    frame2.drop([],axis=0, inplace=True)

#     frame2['sentence_tokens_aslist'] = frame2.sentence.apply(tokenize_aslist)
#     frame2 = frame2.dropna()
#     frame2 = frame2.drop([],axis=0)

#     frame2['sentence_tokens_stemmed_aslist'] = frame2.sentence.apply(tokenize_stem_aslist)
#     frame2 = frame2.dropna()
#     frame2 = frame2.drop([],axis=0)

    return frame, frame2 


import & sanity check

In [11]:
comments = pd.read_csv('/Users/kateliea/Documents/Insight/project/webscrapers/comments_smittenkitchen_all.csv', index_col=0)

In [12]:
comments.columns

Index(['child_id', 'children', 'commentID', 'comment_time', 'recipenumber',
       'title', 'url', 'usercomment', 'username', 'usersite'],
      dtype='object')

## make new dataframe with sentences separated, tokenize everything

In [13]:
comments_only, comments_with_sentences = preprocess_comments_data(comments)

/Users/kateliea/anaconda/envs/py3/lib/python3.5/site-packages/pandas/core/generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/kateliea/anaconda/envs/py3/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/kateliea/anaconda/envs/py3/lib/python3.5/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

NameError: name 'comments_classified' is not defined

In [ ]:
comments_with_sentences.to_csv('comments_with_sentences_all.csv'), comments_only.to_csv('comments_only_all.csv')

In [ ]:
comments_only.shape, comments_with_sentences.shape

In [ ]:
gram = 2
comments_classified.loc[:,'sentence_bigrams'] = comments_classified.loc[:,'sentence'].apply(ngram)
gram = 3
comments_classified.loc[:,'sentence_trigrams'] = comments_classified.loc[:,'sentence'].apply(ngram)

comments_classified.loc[:,'sentence_tokens'] = comments_classified.loc[:,'sentence'].apply(tokenize)
comments_classified.loc[:,'sentence_tokens_stemmed'] = comments_classified.loc[:,'sentence'].apply(tokenize_stem)

comments_classified.dropna(inplace=True)
comments_classified.drop([],axis=0, inplace=True)

comments_classified.to_csv('comments_classified_SK_filtered2000.csv')

In [ ]:
comments_classified.head()